In [1]:
from langchain_ollama import ChatOllama
import json
import re

# TODO
# Usar idioma de la oferta -- OK
# Afinar y aprobar en una segunda pasada la job description para evitar repeticiones, concatenaciones de keywords sin sentido
# traducir titulos de curriculum
# Extraer soft skills generales (evitar usarlas en las job description?)
# usar modelo qwen3:32b 
# optimizar tags html en template
# leer curriculum original para escrapear los datos
# generar pdf
# generar documento editable word? odf? para corregir

cv_file_path = 'data/cv_data_es.json'
soft_skill_file_path = 'data/soft_skill_es.json'
cv_data = {}
soft_skill = {}

wpd_filepath = lambda x: f"workspace/{re.sub(r'[^\w]+', '_', x)}.document.wpd"
cva_filepath = lambda x: f"workspace/{re.sub(r'[^\w]+', '_', x)}.cv_adapted.json"
cv_filepath = lambda x: f"workspace/{re.sub(r'[^\w]+', '_', x)}.cv.html"

# Load JSON file into a dictionary
with open(cv_file_path, 'r') as f:
    cv_data = json.load(f)
with open(soft_skill_file_path, 'r') as f:
    soft_skill = json.load(f)

# with open(soft_skill_file_path, "w") as json_file:
#     json.dump(soft_skill_es, json_file, indent=4)

job_analizer_prompt = """Act as a Job Offer Structure Analizer.
1. Detect the language (e.g Spanish, Catalan, English).
2. Extract the job position title. Use {gender} gender form only.
3. Summarize and synthesize the company description. ensuring that captures the main ideas and values. avoid the corporative jargon. 
4. Separe every single Job function line and add it to the list as it is and, do not elavorate, do no summarice.
5. Extract the job requeriments list as they are, do not elavorate, do no summarice.
/no_think"""

keyword_extractor_prompt = """Extract the keywords in the field of computer science and project management.
Keep the keywords short and simple.
Stick to the text, Do not invent new words.
/no_think"""

task_selector_prompt = """Compare the description given by the user with each description in task_list,
1. Select the id of in task_list that more closely match the description given.
Use semantic similarity analysis to match both context and specific details.
if there is no match just return '-1' and you are done
task_list:

    {task_list}

/no_think"""

# task_selector_prompt = """Find in the task_list the most compatible task.
# if there is no match, call the tool with the task_id=-1 and you are done.
# task_list:

#     {task_list}

# /no_think"""


task_adapter_prompt = """You are a Job Task Adapter.
The user will provide a description of a task in their job.
1. Correct the task description given by using the **keywords**.
2. You can replace original words in the description with keywords as long as the original meaning remains and all together make sense.
3. Check the new description is different to the original description, if not, try again once.
Always use {language} language for the output.
If no keywords provided just translate and return the original.
Syntetize and summarize the in {max_words} words.
high light the **keywords** in **bold**.
keywords: 

    {keywords}

/no_think"""

task_adapter_prompt_no_kw = """Use tool to Return a new_task with the Translation to {language} language of the text.
/no_think"""

skill_extractor_prompt = """Find the soft skills listed in the soft_skills_list in the text and return the finded in a list.
Use the name of the text in the returned list, use {language} language.

soft_skills_list: {soft_skills}

/no_think"""


use_cache=True

llm_strong = ChatOllama(model='qwen3-coder-next:cloud', reasoning=True)
llm_ligth = ChatOllama(model='qwen3:14b', reasoning=False)

job_offer_url = 'https://www.linkedin.com/jobs/view/4362181901'
# job_offer_url = 'https://www.linkedin.com/jobs/view/4317166787'
# job_offer_url = 'https://selection.uoc.edu/web/offersjob/offerdetails.aspx?offerID=5464EA521BB0E0207C9DA853CA6F8505F72B82E3D56D2CAD90882623EBEE1FA7'

In [2]:
from tools import web_operations
import pickle
from langchain_core.documents import Document
import re
import os

def load_document(url, use_cache=True):

    document = None
    job_offer_file_path = wpd_filepath(url)
    if os.path.exists(job_offer_file_path) and use_cache:
        with open(job_offer_file_path, "rb") as f:
            document = pickle.load(f)
            print("loaded from cache file", job_offer_file_path)
    else:
        document = web_operations.scrape_webpages.invoke(input={'url':url, 'include_links':False })[0]
        with open(job_offer_file_path, "wb") as f:
            pickle.dump(document, f)
        print("loaded from url")
        
    return document

document = load_document(job_offer_url, use_cache=use_cache)
print(document.metadata['title'])

loaded from cache file workspace/https_www_linkedin_com_jobs_view_4362181901.document.wpd
DXC Technology hiring Analista técnico/funcional informático/a in Sant Cugat del Vallès, Catalonia, Spain | LinkedIn


In [3]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from typing import Annotated, Literal, TypedDict
from langchain_ollama import ChatOllama
from langfuse.langchain import CallbackHandler

def summarize_text_llm(text: str, 
    llm_model = llm_ligth,
    llm_config = {"configurable": {"thread_id": "resumee_research"}, "recursion_limit": 20, "callbacks": [CallbackHandler()]},
    system_prompt = "Summarize the given text in 100 words /no_think",
    response_format=None,
    name="summarization_agent",
    ) -> str :

    summarization_agent = create_agent(name=name, model=llm_model, system_prompt=system_prompt, response_format=response_format )
    return summarization_agent.invoke({"messages": [("user", text)]}, config=llm_config)


In [4]:
from langchain_core.documents import Document

def job_parts(doc: Document, use_cache=True) -> Document:

    if 'job_position_title' in doc.metadata and use_cache: 
        return doc

    class SummaryOutput(TypedDict):
        language: str
        job_position_title: str
        company_description: str
        # job_description: str
        job_functions: list[str]
        job_requeriments: list[str]

    response = summarize_text_llm(text=f'# {doc.metadata['title']} \n{doc.page_content}', name='job_analizer', llm_model = llm_strong,
        system_prompt = job_analizer_prompt.format(gender=cv_data['gender']),  response_format=ToolStrategy(SummaryOutput))

    doc.metadata['language'] = response['structured_response']['language']
    doc.metadata['job_position_title'] = response['structured_response']['job_position_title']
    doc.metadata['company_description'] = response['structured_response']['company_description']
    # doc.metadata['job_description'] = response['structured_response']['job_description']
    doc.metadata['job_functions'] = response['structured_response']['job_functions']
    doc.metadata['job_requeriments'] = response['structured_response']['job_requeriments']

    with open(wpd_filepath(document.metadata['source']), "wb") as f:
        pickle.dump(doc, f)

    return doc


document = job_parts(document, use_cache=use_cache)
display(document.metadata)

{'source': 'https://www.linkedin.com/jobs/view/4362181901',
 'title': 'DXC Technology hiring Analista técnico/funcional informático/a in Sant Cugat del Vallès, Catalonia, Spain | LinkedIn',
 'description': 'Posted 12:16:42 AM. Job DescriptionJob DescriptionDXC Technology es una compañía global de servicios profesionales cuya…See this and similar jobs on LinkedIn.',
 'language': 'Spanish',
 'job_position_title': 'Analista técnico/funcional informático',
 'company_description': 'DXC Technology es una compañía global de servicios profesionales enfocada en la transformación digital de clientes. Con presencia en más de 70 países y 130.000 profesionales, ofrece soluciones avanzadas de IT como Cloud, IA y automatización. En España es líder del mercado con 8.000 empleados y clientes en sectores públicos y privados. Promueve diversidad e inclusión, con un 49% de mujeres en su plantilla.',
 'job_functions': [{'title': 'Análisis de requisitos y diseño de soluciones informáticas.',
   'keywords': 

In [5]:
# class Keyword(TypedDict):
#     word: str
#     category: Literal['technology_name','soft_skill','organization_name']

class Keywords(TypedDict):
    keywords: list[str]

def extract_keywords(item, use_cache = True):
    keywords = item['keywords'] if type(item) is dict else []
    item = item['title'] if type(item) is dict else item
    if len(keywords) > 0 and use_cache:
        return {'title':item, 'keywords': keywords}
    try:
        response = summarize_text_llm(text=item, name='keyword_extractor', llm_model = llm_ligth,
        llm_config = {"configurable": {"thread_id": "resumee_research"}, "recursion_limit": 4, "callbacks": [CallbackHandler()]},
        response_format=ToolStrategy(Keywords),  system_prompt = keyword_extractor_prompt, )

        keywords = response['structured_response']['keywords']
    except Exception as e:
        print(e)
    return {'title':item, 'keywords': keywords}
  
# extract_keywords(document.metadata['job_requeriments'][0])

In [6]:
def extract_document_keywords(doc: Document, use_cache = True):

    id = 0

    for f in range(0,len(doc.metadata['job_functions'])):
        item = doc.metadata['job_functions'][f]
        item = extract_keywords(item, use_cache = use_cache)
        item['id'] = id
        id = id + 1
        doc.metadata['job_functions'][f] = item
        print(item)

    for f in range(0,len(doc.metadata['job_requeriments'])):
        item = doc.metadata['job_requeriments'][f]
        item = extract_keywords(item, use_cache = use_cache)
        item['id'] = id
        id = id + 1
        doc.metadata['job_requeriments'][f] = item
        print(item)

    with open(wpd_filepath(document.metadata['source']), "wb") as f:
        pickle.dump(doc, f)

    return doc

document = extract_document_keywords(document, use_cache = use_cache)

{'title': 'Análisis de requisitos y diseño de soluciones informáticas.', 'keywords': ['Análisis de requisitos', 'diseño de soluciones informáticas'], 'id': 0}
{'title': 'Identificación de posibles incompatibilidades entre requisitos planteados.', 'keywords': ['requisitos', 'incompatibilidades', 'identificación'], 'id': 1}
{'title': 'ambigüedades e inconsistencias en requisitos.', 'keywords': ['ambigüedades', 'inconsistencias', 'requisitos'], 'id': 2}
{'title': 'Colaborar estrechamente con equipos multidisciplinarios para redactar y revisar propuestas técnicas y comerciales para licitaciones relativas al desarrollo de proyectos informáticos.', 'keywords': ['colaborar', 'equipos multidisciplinarios', 'redactar', 'revisar', 'propuestas técnicas', 'propuestas comerciales', 'licitaciones', 'desarrollo de proyectos informáticos'], 'id': 3}
{'title': 'Asegurando su precisión, claridad y cumplimiento de requisitos.', 'keywords': ['precisión', 'claridad', 'cumplimiento', 'requisitos'], 'id': 4}

In [7]:
def adapt_task(task:str, task_list, language):

    if len(task_list) < 1: return (-1, task)
    
    class MatchesResponse(TypedDict):
        task_id: int

    try:
        response = summarize_text_llm(text = task, llm_model = llm_strong, name='task_selector',
            llm_config = {"configurable": {"thread_id": "resumee_research"}, "recursion_limit": 6, "callbacks": [CallbackHandler()]},
            system_prompt = task_selector_prompt.format(task_list=[{'id':t['id'], 'title':t['title'] } for t in task_list]), response_format=ToolStrategy(MatchesResponse))
       
        task_id = response['structured_response']['task_id']

    except Exception as e:
        print(e)
        task_id = -1

    keywords = [t['keywords'] for t in task_list if t['id'] == task_id]
    system_prompt = task_adapter_prompt_no_kw.format( language=language)
    if len(keywords) > 0: 
        system_prompt = task_adapter_prompt.format( language=language, max_words=len(task.split())*2, keywords=keywords )


    class TaskResponse(TypedDict):
        new_task: str

    response = summarize_text_llm(text = task, llm_model = llm_strong, name='task_adapter',
        llm_config = {"configurable": {"thread_id": "resumee_research"}, "recursion_limit": 6, "callbacks": [CallbackHandler()]},
        system_prompt = system_prompt, 
        response_format=ToolStrategy(TaskResponse))

    return (task_id, response['structured_response']['new_task'])


# adapt_task(task = 'Elaboración de documentación para licitaciones e identificación de requisitos técnicos y funcionales.', 
#     task_list = document.metadata['job_functions']+document.metadata['job_requeriments'],
#     language=document.metadata['language'])


In [8]:
import json

def adapt_cv(cv_data, url, use_cache=True):

    cv_adapted = None
    cv_adapted_file_path = cva_filepath(url)
    if os.path.exists(cv_adapted_file_path) and use_cache:
        with open(cv_adapted_file_path, "rb") as f:
            cv_adapted = json.load(f)
            print("loaded from cache file", cv_adapted_file_path)
    else:
        cv_adapted = dict(cv_data)
        task_list = (document.metadata['job_functions'] + document.metadata['job_requeriments'])
        cv_adapted['title'] = document.metadata['job_position_title']

        for e in range(0, len(cv_adapted['experiences'])):
            for t in range(0, len(cv_adapted['experiences'][e]['description'])):
                orig_task = cv_adapted['experiences'][e]['description'][t]
                try:
                    (task_id, new_task) = adapt_task(task = orig_task, task_list = task_list, language=document.metadata['language'])
                except Exception as ex:
                    print(ex)
                    (task_id, new_task) = (-1, orig_task)    
                base_task = next(iter([t for t in task_list if t['id'] == task_id]), None)
                display({'orig_task':orig_task,
                        'new_task':new_task,
                        'base_task':base_task })
                cv_adapted['experiences'][e]['description'][t] = new_task
                task_list = [t for t in task_list if t['id'] != task_id]
        
        with open(cv_adapted_file_path, "w") as f:
            json.dump(cv_adapted, f, indent=4)

        display(task_list)
        
    display(cv_adapted['experiences'])


    return cv_adapted

cv_adapted = adapt_cv(cv_data, job_offer_url, use_cache=use_cache)


{'orig_task': 'Elaboración de documentación para licitaciones e identificación de requisitos técnicos y funcionales.',
 'new_task': 'Elaboración de **documentación para licitaciones** y **análisis de requisitos**, incluyendo **diseño de soluciones informáticas**.',
 'base_task': {'title': 'Análisis de requisitos y diseño de soluciones informáticas.',
  'keywords': ['Análisis de requisitos', 'diseño de soluciones informáticas'],
  'id': 0}}

{'orig_task': 'Coordinación de equipos técnicos corporativos y seguimiento de proyectos y servicios.',
 'new_task': 'Coordination of corporate technical teams and project and service monitoring.',
 'base_task': None}

{'orig_task': 'Facilitar la comunicación con proveedores externos para garantizar la calidad y cumplimiento de acuerdos.',
 'new_task': 'Facilitar la comunicación con proveedores externos para garantizar la calidad y cumplimiento de acuerdos.',
 'base_task': None}

{'orig_task': 'Impulso de la transformación digital y facilitar la transición a los usuarios hacia nuevas tecnologías.',
 'new_task': 'Translate "Impulso de la transformación digital y facilitar la transición a los usuarios hacia nuevas tecnologías." into English.',
 'base_task': None}

{'orig_task': 'Gestión de presupuestos y ejecución de proyectos según el Plan estratégico.',
 'new_task': 'Gestión de presupuestos y ejecución de proyectos según el Plan estratégico.',
 'base_task': None}

...

{'orig_task': 'Desarrollo de soluciones tecnológicas personalizadas, incluyendo aplicaciones web y sistemas de backend en HTML, CSS, JS, PHP, JAVA.',
 'new_task': 'Desarrollo de soluciones tecnológicas personalizadas, incluyendo aplicaciones web y sistemas de backend en HTML, CSS, JS, PHP y Java.',
 'base_task': None}

{'orig_task': 'Análisis de requisitos y entrevistas con clientes para definir requisitos y entregar soluciones alineadas con sus objetivos. Definición de la base de datos con ORACLE SQL.',
 'new_task': 'Análisis de requisitos e entrevistas con clientes para definir requisitos y entregar soluciones alineadas con sus objetivos. Definición de la base de datos con ORACLE SQL.',
 'base_task': None}

{'orig_task': 'dirigiendo un equipo de tres personas. Nuestro objetivo era el desarrollo de software y automatizaciones para el departamento de operaciones. He desarrollado aplicaciones de gestión, herramientas destinadas al mantenimiento de servidores y diversas automatizaciones para la generación de informes.',
 'new_task': 'Translating the provided Spanish text into English.',
 'base_task': None}

[{'title': 'Identificación de posibles incompatibilidades entre requisitos planteados.',
  'keywords': ['requisitos', 'incompatibilidades', 'identificación'],
  'id': 1},
 {'title': 'ambigüedades e inconsistencias en requisitos.',
  'keywords': ['ambigüedades', 'inconsistencias', 'requisitos'],
  'id': 2},
 {'title': 'Asegurando su precisión, claridad y cumplimiento de requisitos.',
  'keywords': ['precisión', 'claridad', 'cumplimiento', 'requisitos'],
  'id': 4},
 {'title': 'Desarrollar y mantener plantillas y documentación estandarizada para agilizar el proceso de redacción.',
  'keywords': ['desarrollar',
   'mantener',
   'plantillas',
   'documentación',
   'estandarizada',
   'agilizar',
   'redacción'],
  'id': 5},
 {'title': 'Participar en la definición de estrategias y enfoques para licitaciones, considerando las necesidades del cliente y los objetivos comerciales.',
  'keywords': ['licitaciones',
   'estrategias',
   'enfoques',
   'necesidades del cliente',
   'objetivos com

[{'title': 'Gerente de operaciones TIC',
  'location': 'en CTTI (2025)',
  'description': ['Elaboración de **documentación para licitaciones** y **análisis de requisitos**, incluyendo **diseño de soluciones informáticas**.',
   'Coordination of corporate technical teams and project and service monitoring.',
   'Facilitar la comunicación con proveedores externos para garantizar la calidad y cumplimiento de acuerdos.',
   'Translate "Impulso de la transformación digital y facilitar la transición a los usuarios hacia nuevas tecnologías." into English.',
   'Gestión de presupuestos y ejecución de proyectos según el Plan estratégico.',
   'Evaluation and mitigation of risks in ICT services and projects.',
   'Propuesta y desarrollo de iniciativas de mejora continua.',
   'None needed; the text is already in Spanish.',
   'Operational monitoring, incident resolution, and prevention of technological obsolescence.']},
 {'title': 'Gestor de servicios TIC',
  'location': 'en Barcelona Serveis Mu

In [9]:
def extract_skills(content, base_skills_list, language):
    skill_extractor_prompt = """extract the soft skills mentioned the text as they are. do not invent. use tool call to return a skillsResponse"""

# base_skills_list: {base_skills_list}

    class skillsResponse(TypedDict):
        soft_skills: list[str]

    try:
        response = summarize_text_llm(text = content, llm_model = llm_strong, name='skill_extractor',
            # llm_config = {"configurable": {"thread_id": "resumee_research"}, "recursion_limit": 6, "callbacks": [CallbackHandler()]},
            system_prompt = skill_extractor_prompt.format(base_skills_list=base_skills_list, language=language), response_format=ToolStrategy(skillsResponse))

        return response['structured_response']['soft_skills']

    except Exception as e:
        print(e)
        return []


# base_skills_list = [item for value_list in soft_skill.values() for item in value_list]

# extracted_skills = extract_skills(content = document.page_content, base_skills_list=base_skills_list, language=document.metadata['language'])

# extracted_skills

In [11]:
import base64

def encode_base64(data):
    if isinstance(data, str):
        data = data.encode('utf-8')
    
    encoded = base64.b64encode(data)
    return encoded.decode('utf-8')

def decode_base64(encoded_data):
    decoded = base64.b64decode(encoded_data.encode('utf-8'))
    return decoded.decode('utf-8')

# Test encoding
original_text = """secret"""
encoded = encode_base64(original_text)
print(f"Original: {original_text}")
print(f"Encoded:  {encoded}")

# Test decoding
decoded = decode_base64(encoded)
print(f"Decoded:  {decoded}")


Original: secret
Encoded:  c2VjcmV0
Decoded:  secret


In [12]:
from jinja2 import Environment, FileSystemLoader
import re

def render_cv(cv_data):

    # Create environment and load template
    env = Environment(loader=FileSystemLoader('./data/'))
    template = env.get_template('cv_template_es.html')

    cv_data['contact_info'] = decode_base64(cv_data['contact_info'])
    cv_data['name'] = decode_base64(cv_data['name'])

    for e in range(0, len(cv_data['experiences'])):
        for t in range(0, len(cv_data['experiences'][e]['description'])):
            cv_data['experiences'][e]['description'][t] = re.sub(r'\*\*(.*?)\*\*', r'<strong>\1</strong>', cv_data['experiences'][e]['description'][t])

    # Render template with model
    output = template.render(cv_data)

    # Print or save output
    with open(cv_filepath(job_offer_url), "w") as f:
        f.write(output)

render_cv(dict(cv_adapted))